In [65]:
from statsbombpy import sb

df_camp = sb.competitions()
df_camp.head(5)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882


In [78]:
df_matches = sb.matches(competition_id=16, season_id=1)
df_matches.head(3)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,18245,2018-05-26,20:45:00.000,Europe - Champions League,2017/2018,Real Madrid,Liverpool,3,1,available,scheduled,2021-01-23T21:55:30.425330,2021-06-13T16:17:31.694,7,Final,NSK Olimpiyskyi,Milorad Mažić,Zinédine Zidane,Jürgen Klopp,1.1.0,2,2


In [72]:
# Crie uma função simples que receba uma ID de partida e retorne os dados brutos dessa partida utilizando a API.
from statsbombpy import sb

df_events = sb.events(match_id=22912)
df_events.head(3)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,50_50,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,clearance_right_foot,counterpress,dribble_nutmeg,dribble_outcome,dribble_overrun,duel_outcome,duel_type,duration,foul_committed_advantage,foul_committed_offensive,foul_committed_penalty,foul_committed_type,foul_won_advantage,foul_won_defensive,goalkeeper_body_part,goalkeeper_end_location,goalkeeper_outcome,goalkeeper_position,goalkeeper_punched_out,goalkeeper_technique,goalkeeper_type,id,index,interception_outcome,location,match_id,minute,off_camera,out,pass_aerial_won,pass_angle,pass_assisted_shot_id,pass_body_part,pass_cross,pass_cut_back,pass_deflected,pass_end_location,pass_goal_assist,pass_height,pass_inswinging,pass_length,pass_outcome,pass_outswinging,pass_recipient,pass_recipient_id,pass_shot_assist,pass_straight,pass_switch,pass_technique,pass_through_ball,pass_type,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,shot_aerial_won,shot_body_part,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_outcome,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9ec3d009-c15b-4a8d-be80-57a07f40755a,1,NaN,NaN,22912,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,1,Tottenham Hotspur,38,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Tottenham Hotspur,38,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e8bdb5d1-8bcb-4a0d-bfd2-68c18c7f54d0,2,NaN,NaN,22912,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,1,Tottenham Hotspur,38,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'formation': 433, 'lineup': [{'player': {'id'...",Liverpool,24,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00b678a4-3767-4d0b-92e6-9cb4c959d94d,3,NaN,NaN,22912,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,1,Tottenham Hotspur,38,[05840f64-0fb7-4f2d-885a-49790d6bd4df],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Liverpool,24,00:00:00.000,Half Start,NaN


In [21]:
df_events.shot_type.unique()

array([nan, 'Open Play', 'Penalty'], dtype=object)

In [ ]:
# EXERCICIO 3
# Desenvolva uma função que:
#   Identifique os eventos principais da partida (gols, assistências, cartões, etc.).
#   Utilize um modelo LLM para transformar esses eventos em uma sumarização textual descritiva.
#      Tool de leitura de dataframe > https://python.langchain.com/docs/integrations/tools/pandas/
# Certifique-se de que a saída seja clara e amigável, como:
# "O time A venceu o time B por 3 a 1. Os destaques foram os gols de João e Lucas, além de uma assistência de Ana."



colunas_desejadas = set(['second','period','team','player','pass_goal_assist','bad_behaviour_card','shot_type','shot_outcome'])
colunas_no_df = set(df_events.columns)
filtro_colunas = list(colunas_desejadas & colunas_no_df)

# Filtrando o DataFrame para incluir apenas as colunas que estão presentes, pois algumas colunas não aparecem para todas as partidas
X = df_events[filtro_colunas]

melt_values_desejado = set(['pass_goal_assist','bad_behaviour_card','shot_outcome'])
melt_values = list(melt_values_desejado & colunas_no_df)

X = X.melt(id_vars=['second','period','team','player','shot_type'], value_vars=melt_values)
X = X[X.value.notnull()]

descartar = ['Saved','Wayward','Post','Off T','Off S']
X = X[~X['value'].isin(descartar)]

#Concatenar as colunas "Variable" e "Value", usando ponto e virgula como separador

X['key'] = X['variable'] + ';' + X['shot_type'].astype(str) + ';' + X['value'].astype(str) 



#Criando uma coluna única para os eventos
event_map = {
    ('bad_behaviour_card;nan;Yellow Card'): 'Yellow Card',
    ('bad_behaviour_card;nan;Red Card'): 'Red Card',
    ('shot_outcome;Open Play;Blocked'): 'Shot Blocked',
    ('shot_outcome;Open Play;Goal'): 'Regular Goal Scored',
    ('shot_outcome;Penalty;Goal'): 'Penalty Goal Scored',
    ('pass_goal_assist;nan;True'): 'Assist Pass'

}

def combine_variable_value(row):
    return event_map.get(row['key'], 'EVENTO NAO MAPEADO')

# Aplicando a transformação
X['play'] = X.apply(combine_variable_value, axis=1)

#Ordem cronológica
X = X.sort_values(['period','second'])
X

X = X.drop(['variable', 'value','shot_type','key'], axis=1)



df_main_events = X
df_main_events

,second,period,team,player,play
2803,11,1,Liverpool,Jordan Brian Henderson,Shot Blocked
2799,31,1,Liverpool,Mohamed Salah,Shot Blocked
2796,48,1,Liverpool,Mohamed Salah,Penalty Goal Scored
2804,50,1,Liverpool,Mohamed Salah,Shot Blocked
2807,6,2,Liverpool,Mohamed Salah,Shot Blocked
2811,18,2,Tottenham Hotspur,Christian Dannemann Eriksen,Shot Blocked
2806,26,2,Liverpool,Fábio Henrique Tavares,Shot Blocked
2820,39,2,Tottenham Hotspur,Lucas Rodrigues Moura da Silva,Shot Blocked
2808,49,2,Tottenham Hotspur,Bamidele Alli,Shot Blocked
2821,49,2,Liverpool,Virgil van Dijk,Shot Blocked


In [103]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()

google_api_key = os.getenv('GOOGLE_API_KEY')


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=google_api_key
)

# Cria o agente para o DataFrame
agent = create_pandas_dataframe_agent(llm, df_main_events, verbose=True, allow_dangerous_code = True)

# Faz a consulta
response = agent.invoke("As a football commentator, summarize in a paragraph the most important events of the match. Pay attention to the chronological order in the 'second' column and emphasize the teams and players who made each play.")
print(response)




> Entering new AgentExecutor chain...
Thought: I need to sort the dataframe by the 'second' column to get the chronological order of events and then extract the relevant information to create the summary.

Action: python_repl_ast
Action Input: ```python
df_sorted = df.sort_values('second')
print(df_sorted.head())
```      second               team                       player          play
2807       6          Liverpool                Mohamed Salah  Shot Blocked
2803      11          Liverpool       Jordan Brian Henderson  Shot Blocked
2811      18  Tottenham Hotspur  Christian Dannemann Eriksen  Shot Blocked
2806      26          Liverpool       Fábio Henrique Tavares  Shot Blocked
2799      31          Liverpool                Mohamed Salah  Shot Blocked
Thought: Now I have the chronologically sorted dataframe. I can iterate through it and create the summary paragraph.

Action: python_repl_ast
Action Input: ```python
df_sorted = df.sort_values('second')
summary = ""
for index, row

In [ ]:
########## CONTEUDO DO ARQUIVO TOOLS.PY ##########
from langchain.agents import Tool

def instance_lang_chain_tools():
    '''
    Function initializes and returns a list of tools designed for information retrieval.
    '''
    
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    search = DuckDuckGoSearchAPIWrapper()
    
    tools = [
        Tool(
            name="Wikipedia",
            func=wikipedia.run,
            description="Use for fetch general informations and artist history from Wikipedia."
        ),
        Tool(
            name="Internet Search",
            func=search.run,
            description="Use for search current information and news, such as recent releases and tour schedules."
        )
    ]

    return tools

In [44]:
########## CONTEUDO DO ARQUIVO AGENTS.PY ##########

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType
from dotenv import load_dotenv
import os
#from tools import *


load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

if "GOOGLE_CSE_ID" not in os.environ:
    os.environ["GOOGLE_CSE_ID"] = os.getenv('GOOGLE_CSE_ID')

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    verbose=True,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)


def invoke_agent():
    '''
    This function creates an agent that can be used to interact with the user.

    Returns:
    agent_executor: AgentExecutor
    '''
    tools = instance_lang_chain_tools()

    ###### TROCAR PARA MEU NOVO PROMPT##########
    agent_prompt = hub.pull("musicindustrysearch/prompt")
    agent = create_react_agent(llm, tools, agent_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, max_turns=3)
    
    return agent_executor

    ##### NÃO PRECISA ALTERAR NADA AQUI#########
def ask_to_lmm(user_input):
    '''
    This function invoke the agent passing the question on user input.
    Args:
    user_input: str
    '''

    agent = invoke_agent()
    response = agent({"input": user_input})
    return response 

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'tools'

In [ ]:
# EXERCICIO 4
# Desenvolva uma funcionalidade que permita criar um perfil detalhado de um jogador específico da partida, contendo:
#   Nome do jogador.
#   Estatísticas como número de passes, finalizações, desarmes e minutos jogados.
#   Retorne os dados de forma clara e organizada (e.g., JSON ou tabela).

### ACHO QUE VOU FAZER COM SCRIPT E NÃO COM MODELO (VER  O QUE Q O FERNANDO FALOU NA AULA)

In [ ]:
# EXERCICIO 5
# Desenvolvimento de uma API com FastAPI:
#       /match_summary: Retorna a sumarização de uma partida.
#       /player_profile: Retorna o perfil detalhado de um jogador.
# Utilize Pydantic para validar as entradas e saídas.

In [ ]:
# EXERCICIO 6
# Desenvolva uma funcionalidade para criar narrativas baseadas nos eventos da partida.
# Permita ao usuário escolher entre estilos de narração, como:
#       Formal (técnico e objetivo), Humorístico (descontraído e criativo), Técnico (análise detalhada dos eventos)

In [ ]:
#EXERCICIO 7
# Desenvolva uma interface simples que permita ao usuário:
#      Selecionar uma partida por ID.
#      Exibir os eventos principais e o perfil de um jogador.


###### CONTEUDO DO ARQUIVO PRINCIPAL
import streamlit as st
from utils import *

st.set_page_config(layout="wide")

st.sidebar.header('Navegação')

st.sidebar.header('Filtros')
filtros_barra_lateral()



In [ ]:
###### CONTEUDO A COLOCAR NO ARQUIVO UTILS.PY PARA CRIAR A BARRA DE FILTROS

def filtros_barra_lateral():

    #Inicializo as variáveis de sessão
    if 'filtro_camp' not in st.session_state:
        st.session_state.filtro_camp = 'Selecione'

    if 'filtro_ano' not in st.session_state:
        st.session_state.filtro_ano = 'Selecione'

    if 'filtro_partida' not in st.session_state:
        st.session_state.filtro_partida = 'Selecione'

    if 'filtro_id_partida' not in st.session_state:
        st.session_state.filtro_id_partida = 'Selecione'

    if 'df_partidas_filtrado' not in st.session_state:
        st.session_state.df_partidas_filtrado = pd.DataFrame()

    if 'df_eventos' not in st.session_state:
        st.session_state.df_eventos = pd.DataFrame()

    df_camp = sb.competitions()

    #Crio a lista de campeonatos
    lista_camp = df_camp['competition_name'].drop_duplicates().tolist()
    lista_camp.insert(0, 'Selecione')

    #Crio o Seletor
    st.session_state.filtro_camp = st.sidebar.selectbox('Filtrar Campeonato', lista_camp, key='1')

    #Aplico a condição para só filtrar se houver seleção
    if st.session_state.filtro_camp != 'Selecione':
        df_camp = df_camp.loc[df_camp['competition_name'] == st.session_state.filtro_camp]

        #lista dos anos
        lista_ano = df_camp['season_name'].drop_duplicates().tolist()
        lista_ano.insert(0, 'Selecione')
        #seletor de ano
        st.session_state.filtro_ano = st.sidebar.selectbox('Filtrar Ano', lista_ano)

        if st.session_state.filtro_ano != 'Selecione':
            df_camp = df_camp.loc[df_camp['season_name'] == st.session_state.filtro_ano]
            
            #Obtem o Dataset Partidas passando como parâmetro o id do campeonato e o id da temporada    
            df_partidas = sb.matches(competition_id=df_camp['competition_id'].values[0], season_id=df_camp['season_id'].values[0])

            #lista das partidas
            df_partidas['partida'] = df_partidas['home_team'] + ' x ' + df_partidas['away_team']
            df_partidas['resultado'] = df_partidas['home_score'].astype(str) + ' x ' + df_partidas['away_score'].astype(str)
            lista_partidas = df_partidas['partida'].drop_duplicates().tolist()
            lista_partidas.insert(0, 'Selecione')
            #seletor de partidas
            st.session_state.filtro_partida = st.sidebar.selectbox('Filtrar Partida', lista_partidas)

            if st.session_state.filtro_partida != 'Selecione':
                st.session_state.df_partidas_filtrado = df_partidas.loc[df_partidas['partida'] == st.session_state.filtro_partida]

                #Obtem o Dataset Eventos passando como parâmetro o id da partida
                filtro_id_partida = st.session_state.df_partidas_filtrado['match_id'].values[0]
                st.session_state.df_eventos = sb.events(match_id=filtro_id_partida)

    #Exibe um resumo dos filtros selecionados
    texto_selecao = 'ESCOLHA UMA PARTIDA PARA VISUALIZAR'
    
    if st.session_state.filtro_camp != 'Selecione':
    
        st.write('### SELEÇÃO REALIZADA')
        texto_selecao = st.session_state.filtro_camp
        
        if st.session_state.filtro_ano != 'Selecione':
            texto_selecao = st.session_state.filtro_camp + " > " + st.session_state.filtro_ano
            if st.session_state.filtro_partida != 'Selecione':
                texto_selecao = st.session_state.filtro_camp + " > " + st.session_state.filtro_ano + " > " + st.session_state.filtro_partida

    st.write(f'#### {texto_selecao}')

In [ ]:
#EXERCICIO 8
#Implemente um agente LangChain para realizar:
#   Consulta de eventos específicos da partida.
#   Geração de comparações entre dois jogadores.
#   Habilite o agente a responder perguntas como:
#       "Quem deu mais passes na partida?"
#       "Qual jogador teve mais finalizações no primeiro tempo?"

######### ACHO QUE AQUI EU TENHO CRIAR UMA TOOL PERSONALIZADA QUE FAÇA O CALCULO E RETORNE PARA O MODELO  (ASSISTIR A AULA EM QUE O FERNANDO ENSINA A CRIAR UMA TOOL PERSONALIZADA) 
######### LINK ENSINANDO A CRIAR A FERRAMENTA PERSONALIZADA https://python.langchain.com/docs/concepts/tools/

In [ ]:
# EXERCICIO 9
# Melhore a interface Streamlit, adicionando:
#   Filtros para explorar eventos específicos (e.g., apenas gols ou cartões).
#   Visualizações gráficas para estatísticas dos jogadores (e.g., gráficos de barras, tabelas interativas).
# Comparação direta entre dois jogadores selecionados.

############ PEGAR DO TP3, JÁ TEM ISSO PRONTO